In [1]:
import pandas as pd

In [2]:
# read data
data = pd.read_csv("hotel_bookings.csv")

# Intro

I am adding 4 columns: 

Datetime (3)
- day
- week
- month

canceled_days (1)

The purpose of these columns are to help evaluate different slices of the results (i.e different time-periods). They are not meant to help increase the models performance (and they won't be included in training)

# datetime: daily, weekly, monthly

creating new date-columns based on existing date-columns

Existing cols have year, month and week separate and I want columns for day, week and month with year included. I find this more efficient and clear when looking at different time periods as I don't have to deal with multiindex.

## daily

In [32]:
# add new column with datetime for day of arrival
arrival_date = pd.to_datetime(data.arrival_date_year.astype(str) + "-" + data.arrival_date_month.astype(str) + "-" + data.arrival_date_day_of_month.astype(str)).tolist()
data["arrival_date"] = arrival_date

## weekly

In [19]:
# making them into strings to be able to add them together
# removing "W" from week column so datetime can read it
data["year_week"] = data.arrival_date_year.astype(str) + "-W" + data.arrival_date_week_number.astype(str)

In [15]:
# adding "-1" to get first day for week.
# "%Y-W%W-%w" instructs formating 
import datetime
def year_we(x):
    #print(x)
    r = datetime.datetime.strptime(x + '-1', "%Y-W%W-%w")
    #print(r)
    return r

In [16]:
# applying function
data.year_week = data.year_week.apply(year_we)

## monthly

In [21]:
# creating year and month column
# straightforward, datetime can parse even if only year and month is specified, it just puts the first day for each month
data["year_mo"] = pd.to_datetime(data.arrival_date_year.astype(str) + "-" + data.arrival_date_month.astype(str))

# days before canceling

I am creating a new new column where I store number of days before arrival that a booking was canceled. This column is thus only relevant for bookings that were canceled. 

To give the reader a feel for the utility of this feature: 

When evaluating the models performance I might (for a particular problem) only be interested in predictions made for bookings that will cancel within 30-60 days of arrival (imagine that the hotel is doing its resource-planning on a monthly basis)

## For canceled_bookings

In [37]:
# Changing format to datetime for reservataion_status_date (date when booking was canceled)
data.reservation_status_date = pd.to_datetime(data.reservation_status_date)

In [38]:
# Getting the no. of days before arrival_date that the booking was canceled
data["canceled_days"] = data.arrival_date - data.reservation_status_date

In [39]:
# function to get only an integer for number of days instead of timestamp-format...
# ...an integer is much more convenient to look at rather then a timestamp
import re

def no_days(x):
    x = str(x)
    r = re.findall(r"[1-9]+", x)
    if r:
        return r[0]
    else:
        return 0

In [40]:
# apply function to only get integers for number of days
data.canceled_days = data.canceled_days.apply(no_days)

## N/A for bookings that weren't canceled

In [41]:
# replace values for bookings that were not canceled with NA since the number otherwise can be misleading
data.iloc[data.loc[data.is_canceled == 0].index.tolist(), 35] = "NA"

# Writing to csv

In [43]:
# creating new csv with the added columns/features
data.to_csv("hotel_clean.csv")